# embed-text-doc2vec

based on https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

first, let's install some dependencies. a guide to doing this: https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

In [1]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} gensim nltk

Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/m/anaconda3/envs/parse-html:
#
gensim                    2.3.0               np113py36_0  
nltk                      3.2.4                    py36_0  


In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

Let’s prepare data for training our doc2vec model

In [2]:
data_dir = '../../data/'

# our list of documents
data = []

In [3]:
import glob
txt_files = glob.glob(f"{data_dir}/*.txt")
print(len(txt_files))

141


In [4]:
# should an example of just the filename without the path
txt_files[0][11:]

'Trieu et al. - 2017 - News Classification from Social Media Using Twitte.txt'

In [5]:
for file in txt_files:
    with open(file, 'r', encoding="utf-8") as file:
        currentText = file.read()
        data.append(currentText)
        file.close()

print(len(data))

141


In [6]:
from random import randrange
random_index = randrange(len(data)-1)

# print the first 1000 characters of a random document from our corpus
print(data[random_index][0:1000])

This article has been accepted for publication in a future issue of this journal, but has not been fully edited. Content may change prior to final publication. Citation information: DOI 10.1109/TKDE.2016.2557324, IEEE
Transactions on Knowledge and Data Engineering
IEEE TRANSACTIONS ON KNOWLEDGE AND DATA ENGINEERING, VOL. 11, NO. 12, 2015

1

Interactive Visualization
of Large Data Sets
Parke Godfrey ∗ , Jarek Gryz ∗ , Piotr Lasek ∗ †
{godfrey, jarek, plasek}@cse.yorku.ca
York University, Canada ∗
Rzeszów University, Poland †
Abstract—Visualization provides a powerful means for data analysis. But to be practical, visual analytics tools must support smooth
and flexible use of visualizations at a fast rate. This becomes increasingly onerous with the ever-increasing size of real-world datasets.
First, large databases make interaction more difficult once query response time exceeds several seconds. Second, any attempt to show
all data points will overload the visualization, resulting in ch

In [7]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), 
    tags=[str(i)]) for i, _d in enumerate(data)]

Here we have a list of four sentences as training data. Now I have tagged the data and its ready for training. Lets start training our model.

In [8]:
max_epochs = 100
vec_size = 20
alpha = 0.025
model = Doc2Vec(size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm=1)

In [9]:
model.build_vocab(tagged_data)

In [10]:
for epoch in range(max_epochs):
    print ('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("d2v.model")
print("Model d2v.model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

Note: dm defines the training algorithm. If dm=1 means ‘distributed memory’ (PV-DM) and dm =0 means ‘distributed bag of words’ (PV-DBOW). Distributed Memory model preserves the word order in a document whereas Distributed Bag of words just uses the bag of words approach, which doesn’t preserve any word order.

So we have saved the model and it’s ready for implementation. Lets play with it.

In [11]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")

#to find the vector of a document which is not in the training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)




V1_infer [-0.05315243  0.15070046  0.0249238  -0.11311816 -0.03199848 -0.04710091
 -0.03913097  0.08789978 -0.02645869 -0.00255124  0.15403078 -0.16651161
  0.17035538  0.01202741 -0.00514478 -0.08900966  0.00446956  0.15910985
 -0.03648923 -0.00031103]


In [12]:
# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)

[('70', 0.9870254397392273), ('84', 0.9709815979003906), ('7', 0.96864914894104), ('94', 0.960952639579773), ('72', 0.9577339291572571), ('14', 0.9566090106964111), ('18', 0.953985333442688), ('82', 0.9523268938064575), ('97', 0.9513706564903259), ('0', 0.9446595907211304)]


In [13]:
# to find vector of doc in training data using tags
# or in other words printing the vector of the document 
# at index 1 in the training data
print(model.docvecs['1'])

[ -9.342972  -17.231485  -16.349152   17.070536    8.355379   -3.7523832
  13.2250395 -11.772013    6.22972    11.040406  -12.797823   -1.3005092
 -17.47004    -5.203946    4.065793   -2.5147984   5.3141646 -10.981502
  16.272821   -1.7505883]


In [14]:
# how many dimensions does our doc2vec document space have?
dimensions = len(model.docvecs['1'])
print(dimensions)

20


Cool! This dimensionality is determined by the `vec_size` parameter we specified at training time.

In [15]:
# create column headers for csv file
headers = ['doc']
i = 0
while i < dimensions:
    headers.append(f"v{i}")
    i+=1
    
print(headers)

['doc', 'v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19']


In [16]:
# retrieve vectors of all documents in training data
# write vectors to a csv file
import csv

with open('document-vectors.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"')
    writer.writerow(headers)
    
    index_count = len(data)-1
    i = 0
    while i <= index_count:
        doc_name = txt_files[i][11:]
        vec = list(model.docvecs[i])
        row = [doc_name] + vec
        writer.writerow(row)
        i += 1


In [56]:
# read vectors in from csv file
import csv

imported_vectors = []

with open('document-vectors.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        imported_vectors.append(row)
        
print(imported_vectors[0:2])

[['doc', 'v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19'], ['Trieu et al. - 2017 - News Classification from Social Media Using Twitte.txt', '37.8572', '9.51042', '-12.9113', '-15.5441', '33.3247', '-21.667', '-26.5227', '5.00173', '-6.14793', '48.2965', '50.8205', '15.4903', '-5.50221', '-20.1339', '-38.5444', '5.88733', '6.06908', '9.33432', '-25.3571', '-12.2529']]


In [5]:
# project from 20D to 2D with t-SNE
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [6]:
model.docvecs

NameError: name 'model' is not defined

In [7]:
df = pd.DataFrame(list(model.docvecs))

def stripPath(file):
    return file[11:]

# list(map(stripPath, txt_files))
df['y'] = list(range(0,len(list(model.docvecs))))
df['label'] = df['y'].apply(lambda i: str(i))

# X, y = None, None

print('Size of the dataframe: {}'.format(df.shape))

NameError: name 'model' is not defined

In [42]:
# For reproducibility of the results
np.random.seed(42)

rndperm = np.random.permutation(df.shape[0])

In [43]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(df)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 141 samples in 0.000s...
[t-SNE] Computed neighbors for 141 samples in 0.003s...
[t-SNE] Computed conditional probabilities for sample 141 / 141
[t-SNE] Mean sigma: 26.328567
[t-SNE] KL divergence after 250 iterations with early exaggeration: 51.034737
[t-SNE] KL divergence after 300 iterations: 0.137496
t-SNE done! Time elapsed: 0.26351213455200195 seconds


In [4]:
# visualize t-SNE projection
results = pd.DataFrame()

results['tsne-2d-one'] = tsne_results[:,0]
results['tsne-2d-two'] = tsne_results[:,1]

# plt.figure(figsize=(16,10))
# sns.scatterplot(
#     x="tsne-2d-one", y="tsne-2d-two",
#     hue="y",
#     palette=sns.color_palette("hls", 10),
#     data=results,
#     legend="full",
#     alpha=0.3
# )

plt.figure(figsize=(16,10))
sns.lmplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=results,
    legend="full",
    alpha=0.3,
    fit_reg=False,
    scatter_kws={"marker": "D", # Set marker style
                           "s": 100} # S marker size
)

NameError: name 'tsne_results' is not defined

In [38]:
# project from 20D to 2D with UMAP

In [33]:
# visualize UMAP projection